# Number of lines in file

```
data_file = '/Volumes/seagate-storage/code/ebird_data/ebd_relJan-2019.txt'
count = 0
for line in open(data_file): count += 1
```

After this, `count` = 577335302

# Figure out which columns to store

Save:
* `LAST EDITED DATE`
* `CATEGORY`
* `COMMON NAME`
* `OBSERVATION COUNT`
* `COUNTY CODE`
* `LATITUDE`
* `LONGITUDE`
* `OBSERVATION DATE`
* `TIME OBSERVATIONS STARTED`
* `OBSERVER ID`
* `SAMPLING EVENT IDENTIFIER`
* `PROTOCOL TYPE`
* `DURATION MINUTES`
* `NUMBER OBSERVERS`
* `ALL SPECIES REPORTED`


In [1]:
%%bash
head -2 /Volumes/seagate-storage/code/ebird_data/ebd_relJan-2019.txt

GLOBAL UNIQUE IDENTIFIER	LAST EDITED DATE	TAXONOMIC ORDER	CATEGORY	COMMON NAME	SCIENTIFIC NAME	SUBSPECIES COMMON NAME	SUBSPECIES SCIENTIFIC NAME	OBSERVATION COUNT	BREEDING BIRD ATLAS CODE	BREEDING BIRD ATLAS CATEGORY	AGE/SEX	COUNTRY	COUNTRY CODE	STATE	STATE CODE	COUNTY	COUNTY CODE	IBA CODE	BCR CODE	USFWS CODE	ATLAS BLOCK	LOCALITY	LOCALITY ID	 LOCALITY TYPE	LATITUDE	LONGITUDE	OBSERVATION DATE	TIME OBSERVATIONS STARTED	OBSERVER ID	SAMPLING EVENT IDENTIFIER	PROTOCOL TYPE	PROTOCOL CODE	PROJECT CODE	DURATION MINUTES	EFFORT DISTANCE KM	EFFORT AREA HA	NUMBER OBSERVERS	ALL SPECIES REPORTED	GROUP IDENTIFIER	HAS MEDIA	APPROVED	REVIEWED	REASON	TRIP COMMENTS	SPECIES COMMENTS
URN:CornellLabOfOrnithology:EBIRD:OBS34169564	2014-10-16 08:19:55	6171	species	Bonaparte's Gull	Chroicocephalus philadelphia			6				Canada	CA	Quebec	CA-QC	Nicolet-Yamaska	CA-QC-NY					Port St-François	L352699	P	46.2722222	-72.6111111	2006-10-14	10:00:00	obsr31597	S2495584	Traveling	P22	EBIRD_CAN	60	1		1	0		0	1	0		Terns are lat

In text format:

```
GLOBAL UNIQUE IDENTIFIER	                    LAST EDITED DATE	        TAXONOMIC ORDER	    CATEGORY	    COMMON NAME	        SCIENTIFIC NAME	               SUBSPECIES COMMON NAME	SUBSPECIES SCIENTIFIC NAME	OBSERVATION COUNT	BREEDING BIRD ATLAS CODE	BREEDING BIRD ATLAS CATEGORY	AGE/SEX	COUNTRY	COUNTRY CODE	STATE	STATE CODE	COUNTY	            COUNTY CODE	IBA CODE	BCR CODE	USFWS CODE	ATLAS BLOCK	LOCALITY	        LOCALITY ID	LOCALITY TYPE	LATITUDE	LONGITUDE	OBSERVATION DATE	TIME OBSERVATIONS STARTED	OBSERVER ID	SAMPLING EVENT IDENTIFIER	PROTOCOL TYPE	PROTOCOL CODE	PROJECT CODE	DURATION MINUTES	EFFORT DISTANCE KM	EFFORT AREA HA	NUMBER OBSERVERS	ALL SPECIES REPORTED	GROUP IDENTIFIER	HAS MEDIA	APPROVED	REVIEWED	REASON	TRIP COMMENTS	SPECIES COMMENTS
URN:CornellLabOfOrnithology:EBIRD:OBS34169564	2014-10-16 08:19:55	        6171	            species	        Bonaparte's Gull	Chroicocephalus philadelphia		                    	                        6	                	                        	                        	        Canada	CA	            Quebec	CA-QC	     Nicolet-Yamaska	CA-QC-NY		        	        	        	        Port St-François	L352699	    P	            46.2722222	-72.6111111	2006-10-14	         10:00:00	                obsr31597	S2495584	                Traveling	    P22	            EBIRD_CAN	    60	                1	                	            1	                0	                    	                0	        1	        0	        	    Terns are late!	
```
                    
                    

# Store a small subset of text file

In [2]:
import pandas as pd
from time import time
from sqlalchemy import create_engine, MetaData
from sqlalchemy_utils import database_exists, create_database
from functools import wraps

### List of columns for each type of database

In [3]:
desired_cols_comprehensive = [
    'LAST EDITED DATE', #For telling whether or not the observation was submitted in 2018
    'CATEGORY', #Species or no? (not sure how others are represented)
    'COMMON NAME', 
    'OBSERVATION COUNT', #Number of individuals seen--or X'd (not sure how represented)
    'STATE CODE', #Something like US-PA-AL. Not sure if every obs has a county...
    'LATITUDE',
    'LONGITUDE',
    'OBSERVATION DATE',
    'TIME OBSERVATIONS STARTED',
    'OBSERVER ID',
    'SAMPLING EVENT IDENTIFIER', #The checklist itself... Not sure if there are anything besides S_____ formatted labels, and what the difference is from global unique identifier (maybe the latter is for the single species obs)
    'PROTOCOL TYPE', #Traveling... or something else? Not sure what else is out there
    'DURATION MINUTES', #To see if it has both a start and end time
    'NUMBER OBSERVERS', #Not sure if each observer's list is counted as a separate observation, or if only the originally submitting observer's is included
    'ALL SPECIES REPORTED',
]

# A subset of desired_cols_comprehensive; pared down for speed
desired_cols_chaser = {
    'CATEGORY':'str', #Species or no? (not sure how others are represented)
    'COMMON NAME':'str',
    'OBSERVATION COUNT':'str', #Number of individuals seen--or X'd (not sure how represented)
    'STATE CODE':'str', #Something like US-PA-AL. Not sure if every obs has a county...
    'LATITUDE':'str',
    'LONGITUDE':'str',
    'OBSERVATION DATE':'str',
    'SAMPLING EVENT IDENTIFIER':'str', #The checklist itself... Not sure if there are anything besides S_____ formatted labels, and what the difference is from global unique identifier (maybe the latter is for the single species obs)
}

### Variables for `pd.read_csv` call:

In [4]:
# File to read from & number of lines to read
#data_file = '/Volumes/seagate-storage/code/ebird_data/medium_dataset.txt']
large_data_file = '/Volumes/seagate-storage/code/ebird_data/ebd_relJan-2019.txt'
data_file = '/Volumes/seagate-storage/code/ebird_data/small_file.txt'
lines_to_use = 1000000

# Made data file using terminal instead of Python, but here's the code I used for smaller tests.
with open(large_data_file, 'r') as oldfile:
    with open(data_file, 'w') as newfile: #Overwrite old file
        for _ in range(lines_to_use):
            newfile.write(next(oldfile))

# Which columns to use
desired_cols = desired_cols_chaser
                          
# Of columns, which are dates?
date_cols = ['OBSERVATION DATE']

In [5]:
# Chunk size to read through data_file
chunk_size = 10000

### Variables for `pd.to_sql` call:

In [6]:
# Desired path for SQLITE db; if does not exist, will be created
engine_path = "/Volumes/seagate-storage/db/ebird_small.db"

# Name of SQL table
table_name = 'ebird_small_20190328'

### Other variables

In [9]:
# How many chunks to print after
X = 100

### Finally, save to database:

In [10]:
def process_data(
    engine_path,
    data_path,
    desired_cols,
    date_cols,
    chunk_size,
    table_name,
    X,
    delete_old_table = True
):
    t0 = time()
    
    engine = create_engine('sqlite:///' + engine_path)
    # If database doesn't already exist, make a new engine
    if not database_exists(engine.url):
        create_database(engine.url)
    # Otherwise, make sure the engine is clear
    # Note that it's MUCH faster to delete this from filesystem directly
    elif delete_old_table:
        meta = MetaData(engine)
        meta.reflect() # Find all tables
        # Only drop the table we're trying to refill
        for tbl in reversed(meta.sorted_tables):
            if tbl.name == table_name:
                engine.execute(tbl.delete())
                print(f'Dropped table {table_name}')
        
    t1 = time()
    
    lines_saved_to_db = 0
    counter = 0

    for chunk in pd.read_csv(
        filepath_or_buffer = data_path, 
        sep = '\t',
        usecols = desired_cols.keys(),
        parse_dates = date_cols, 
        chunksize = chunk_size,
        dtype = desired_cols,
        na_values = ['X']
    ):

        chunk.to_sql(
            name = table_name,
            con = engine,
            if_exists = 'append'
        )

        # Print info every X iterations
        # i.e., every chunk_size * X lines
        lines_saved_to_db += chunk.shape[0]
        if (not counter % X):
            print(f"Iteration {counter}:")
            print(f"  Lines saved to DB: {lines_saved_to_db}") #Helpful for only saving year data
            print(f"  Minutes elapsed: {(time()-t1)/60}\n")
        counter += 1

    db_time = (time()-t1)/60
    total_time = (time()-t0)/60
    estimated_time = ((total_time-db_time) +  (577335302/lines_saved_to_db)*db_time)/60
    print(f"Total iterations: {counter}")
    print(f"Total lines saved to DB: {lines_saved_to_db}") #Helpful for only saving year data
    print(f"Minutes elapsed during DB functions: {db_time}")
    print(f"Total minutes elapsed: {total_time}")
    print(f"Estimated time for all lines: {estimated_time} hours")
    return engine
            
engine = process_data(
    engine_path,
    data_file,
    desired_cols,
    date_cols,
    chunk_size,
    table_name,
    X
)


Dropped table ebird_small_20190328
Iteration 0:
  Lines saved to DB: 10000
  Minutes elapsed: 0.006281014283498128

Total iterations: 100
Total lines saved to DB: 999999
Minutes elapsed during DB functions: 0.41043624877929685
Total minutes elapsed: 0.4188800017038981
Estimated time for all lines: 3.9494669392206614 hours


# Try chaser on subset

In [11]:
import csv
import pandas as pd
import numpy as np
from geopy.distance import vincenty
import datetime
import collections
import math

Done below:
```
show_unique(series)
identify_centers(df, radius_miles=5)
select_loc_df(df, my_dict, center)
select_date_df(df, month, day, interval=4)
calculate_score(df)
interval_without_year(date1, date2)
analyze_data(
    df,
    lifelist_path,
    month,
    day,
    radius_miles)
def import_data(
    engine_path,
    table_name)
```

In [20]:
def show_unique(series):
    '''
    Return a list of the unique elements in a pandas series
    '''
    
    all_elements = list(series)
    seen = set()
    seen_add = seen.add
    unique_elements = [x for x in all_elements if not (x in seen or seen_add(x))]

    return unique_elements

In [31]:
def identify_centers(df, radius_miles = 5):
    '''
    Create a dictionary where the
        keys = centers,
        values = list of points within the desired radius
    '''
    
    # Create a list of all unique points to use for our centers
    all_points = show_unique(df['LOCATION'])
    centers = all_points #TODO: create a more efficient center-finding algo

    my_dict = {}
    centers = all_points
    for center in centers:
        close_points = []
        for point in all_points:
            try:
                distance = vincenty(center, point, miles=True)
            except ValueError: #ValueError: Vincenty formula failed to converge!
                distance = np.NaN
            if distance < radius_miles:
                close_points.append(point)

        my_dict[center] = close_points
        
    return my_dict

In [22]:
def select_loc_df(df, my_dict, center): 
    '''
    Select relevant sightings by location
    
    For a single center (key), return a DF with only:
    - sightings from locations in the desired radius (value) 
    - sightings from a desired date range
    
    Inputs:
        df: the full dataframe to select sightings from
        my_dict: the dictionary associating centers with lists
            of points within the radius
        center: the specific dictionary key to use
    
    '''

    # Select only locations corresponding to radius around the center
    return df[df['LOCATION'].isin(my_dict[center])]

In [23]:
def select_date_df(df, month, day, interval=4): 
    '''
    Select relevant sightings by date
    
    For a single center (key), return a DF with only sightings from a desired date range
    
    Inputs:
        df: the full dataframe to select sightings from
        month, day: the date to center a date range around
        interval: the number of days before & after the specified
            date to include sightings from
    
    '''
    
    # Select only locations within date range
    desired_date=datetime.date(2019, month, day) #2019 is a placeholder year

    df['INTERVAL'] = df['OBSERVATION DATE'].apply(interval_without_year, date2=desired_date)
    selected_sightings = df.loc[df['INTERVAL'] <= interval]
    
     
    return selected_sightings

In [67]:
def calculate_score_total_spp(df):
    '''
    Calculate score of DF based on # species
    
    Calculate score of a DF solely by how many
    unique species are in the 'COMMON NAME' column.
    
    Inputs:
        df: a dataframe with a column 'COMMON NAME'. This df should
        contain one column for each individual observation of a species.
        
    Returns: 
    '''
    
    # Create DF associating common name with count of lists
    #(to maintain compliance with other scoring functions)
    collapsed_df = df['COMMON NAME'].value_counts()
    
    # Calculate true score
    score = len(show_unique(df['COMMON NAME']))
    
    return score, collapsed_df

In [68]:
def calculate_score_num_lists(df):

    '''
    Calculate score of DF based on # lists
    
    Calculate the score of a DF by finding
    the number of checklists on which a species was reported.
    This could be modified to be more sophisticated in the future.
    
    Inputs:
        df: a dataframe with a column 'COMMON NAME'. This df should
            contain one column for each individual observation of a species.
    
    Outputs:
        the sum of the number of checklists each species
        appeared on--which is identical to df.shape[0]
    '''
    
    # Create DF associating common name with count of lists
    collapsed_df = df['COMMON NAME'].value_counts()
    
    # Calculate true score
    score = sum(collapsed_df)
    
    return score, collapsed_df

In [25]:
def interval_without_year(date1, date2):
    '''
    Return number of days between two dates
    
    Returns interval of days between two dates regardless of year. 
    For instance, January 1, 2019 and January 4, 1999 are considered 
    3 days apart with this method, no matter which one is provided
    as day1 or day2.
    
    This function may be imperfect WRT leap years.
    
    Inputs:
        date1, date2: datetime.date objects
        
    Returns:
        number of days between them on calendar (regardless of year)
    
    '''
    date1 = date1.date()
    delta = date1-date2
    way1 = delta.days % 365
    way2 = 365-way1
    
    if way1 < way2: return way1
    else: return way2

In [111]:
def analyze_data(
    df,
    lifelist_path,
    month,
    day,
    padding,
    radius_miles,
    scoring_method = 'total' #or 'lists'
):
    '''
    Find best area for a given month, day, and radius
    
    Inputs
        df: dataframe of eBird observations generated by import_data
        lifelist_path: path to .csv of life list file. Species should be in format:
            'Black-bellied Whistling-Duck - Dendrocygna autumnalis' (without quotes)
        month: numerical month of 'center date' around which to search for hotspots
        day: numerical day of 'center date' around which to search for hotspots
        padding: how many days before and after the 'center date' the observations should include
        radius_miles: how large of a radius do you want to search within
        scoring_method: how to score hotspots
            total: just based on the total number of species
            lists: based on how many lists each lifer appeared
        
    Returns (all_dfs, all_scores)
        all_dfs: a dataframe for every center
        all_scores: a score for every center
    '''
    
    # Generate list of lifers from .csv file
    life_list = []
    with open(lifelist_path) as f:
        reader = csv.reader(f)
        
        # Handle different eBird life list formats
        next(reader) # Skip line 0, the header
        test_bird = next(reader)[1]
        f.seek(1) # Go back to line 1, the first data row
        # For scientific names
        if ' - ' in test_bird:
            for line in reader:
                bird = ' '.join(line[1].split()[:-3])
                life_list.append(str(bird).lower()) 
                
        # For no scientific names:
        else:
            for line in reader:
                bird = line[1]
                life_list.append(str(bird).lower()) 
        
        

    # Remove species that are already on one's life list
    df = df[~df['COMMON NAME'].str.lower().isin(life_list)]
    
    
    # Remove checklists from outside of this date range
    df = select_date_df(df, month, day, interval=4)
    
    
    # Identify a dict of centers
    my_dict = identify_centers(df, radius_miles)
    
    
    # For each center, generate a relevant DF and calculate its score
    all_dfs = {}
    all_scores = {}
    for center in my_dict.keys():
        
        # Select only locations within radius
        selected_df = select_loc_df(df, my_dict, center)
        
        # Calculate score and 'collapsed DF', which is a dataframe
        # associating each species with the number of checklists it appeared on
        if scoring_method == 'lists':
            score, collapsed_df = calculate_score_num_lists(selected_df)
        else:
            score, collapsed_df = calculate_score_total_spp(selected_df)
        
        # Create dictionaries associating centers with their scores & collapsed dfs
        all_scores[center] = score
        all_dfs[center] = collapsed_df
        
    
    return (all_dfs, all_scores)

In [100]:
def import_data(
    engine_path,
    table_name,
):
    '''
    Import and clean dataset from database.
    
    Import a dataset from a database. Add a location column and remove
    extraneous records (spuhs, slashes, hybrids, domestics) from dataset.
    
    Inputs
        engine_path: path to db location, e.g. '/Volumes/storage/data.db'
        table_name: the name of the table in this database, e.g. 'usa_only'
        
    Returns
        a rough database
    '''
    
    # Import data
    engine = create_engine('sqlite:///' + engine_path)
    rough_df = pd.read_sql(table_name, con = engine)
    
    # Add location column
    rough_df['LOCATION'] = rough_df.apply(lambda row: (row['LATITUDE'], row['LONGITUDE']), axis=1)
    
    # Remove spuhs, slashes, domestics, and hybrids to leave only species, issf, and form
    return rough_df[~rough_df['CATEGORY'].isin(['spuh', 'domestic', 'slash', 'hybrid'])]


In [126]:
def chaser(
    engine_path,
    table_name,
    lifelist_filename,
    month,
    day,
    padding,
    num_spots,
    radius_miles,
    scoring_method = 'total' #or 'lists'
):
    
    '''
    Find the best places to go for your life list
    
    Inputs
        engine_path: path to database of all observations, e.g. '/Volumes/storage/all_data.db'
        table_name: name of relevant table within database, e.g. 'usa_only_table'
        lifelist_filename: path to .csv file containing a single column
            All species must be in format: 'Black-bellied Whistling-Duck - Dendrocygna autumnalis'
            To create a file like this, download your life list, then copy and paste the 'Species'
            column into a new .csv. Do not include the 'species' header (though it won't make a difference).
            Life list can be downloaded at https://ebird.org/MyEBird?cmd=lifeList&listType=world&listCategory=default&time=life
        month: numerical month in which to search for lifer spots
        day: numerical day around which to search for lifer spots
        padding: number of days before and after month/day to search for lifer spots
        num_spots: number of top spots to show
        radius_miles: miles within which you are willing to bird
    '''
    
    
    # Get DF of data
    my_df = import_data(engine_path, table_name)
    
    # Only look for places in the USA (#ABAcentrism)
    my_df = my_df[(my_df['STATE CODE'].str.contains('US-')) | (my_df['STATE CODE'].str.contains('CA-'))]

    # Get dict of dfs of highest-scoring spots
    dfs, scores = analyze_data(
        df = my_df,
        lifelist_path = lifelist_filename, 
        month = month,
        day = day,
        padding = padding,
        radius_miles = radius_miles,
        scoring_method = scoring_method)


    # Sort by scores
    #sorted_d = sorted(((value, key) for (key,value) in scores.items()), reverse=True)
    
    # Sort by number of targets
    sorted_d = sorted(((value, key) for (key,value) in scores.items()), reverse=True)

    if len(sorted_d) < num_spots:
        num_to_show = len(sorted_d)
    else:
        num_to_show = num_spots

    for i in range(num_to_show):
        center =  sorted_d[i][1]

        # Identify most common targets
        center_df = dfs[center]
        top_10percent = math.ceil(.1 * center_df.size)
        targets = list(center_df.reset_index()['index'][0:])

        print('Top spot {}:'.format(i+1), center)
        print('Number possible targets:', len(targets))
        print('Most common targets:', targets)
        print('')
    
    return scores

# Washington yearlist, J

In [127]:
engine_path = "/Volumes/seagate-storage/db/ebird_small.db"
table_name = 'ebird_small_20190328'
lifelist_filename = 'josh-yearlist.csv'
month = 3
day = 29
padding = 2
num_spots = 5
radius_miles = 100

josh_scores_all = chaser(
    engine_path,
    table_name,
    lifelist_filename,
    month,
    day,
    padding,
    num_spots,
    radius_miles
)

josh_scores_all

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.


Top spot 1: ('27.0446212', '-99.4444013')
Number possible targets: 79
Most common targets: ['Chihuahuan Raven', 'Northern Cardinal', 'Great Kiskadee', 'Northern Mockingbird', 'Scissor-tailed Flycatcher', 'Golden-fronted Woodpecker', 'Altamira Oriole', 'Black-crested Titmouse', 'Great-tailed Grackle', 'Loggerhead Shrike', 'Lark Sparrow', 'Inca Dove', "Harris's Hawk", 'Laughing Gull', 'White-tipped Dove', "Audubon's Oriole", 'Plain Chachalaca', "Couch's Kingbird", 'Common Ground-Dove', 'Red-billed Pigeon', 'Cedar Waxwing', 'Spotted Sandpiper', 'Crested Caracara', 'Northern Rough-winged Swallow', 'Green Kingfisher', 'Purple Martin', 'Chimney Swift', 'Mottled Duck', 'Blue-gray Gnatcatcher', 'Brown-headed Cowbird', "Swainson's Hawk", 'Ringed Kingfisher', 'Barn Swallow', 'Cliff Swallow', 'Neotropic Cormorant', 'White-winged Dove', 'Curve-billed Thrasher', 'Common Yellowthroat', "Lincoln's Sparrow", 'Ladder-backed Woodpecker', 'Broad-winged Hawk', 'Lesser Nighthawk', 'Greater Roadrunner', 'Ci

{('46.6944318', '-60.3571701'): 6,
 ('41.5851223', '-70.8219668'): 23,
 ('41.9351042', '-82.5104141'): 21,
 ('46.2722222', '-72.6111111'): 25,
 ('46.65', '-71.566667'): 21,
 ('21.9549272', '-159.335736'): 25,
 ('38.0725405', '-122.8141122'): 32,
 ('46.7561383', '-71.2436413'): 15,
 ('43.3130395', '-77.7153471'): 18,
 ('42.687714', '-70.6204365'): 45,
 ('27.367185', '-82.626513'): 18,
 ('43.8993331', '-77.2260182'): 20,
 ('41.9435608', '-87.636194'): 39,
 ('37.7542941', '-122.2506237'): 53,
 ('21.2834562', '-157.7271429'): 7,
 ('48.06667', '-65.56667'): 5,
 ('29.4833333', '-94.5166667'): 28,
 ('58.5175483', '-134.8334885'): 4,
 ('44.8342', '-87.377'): 7,
 ('41.2636142', '-72.561779'): 31,
 ('46.8', '-71.18333'): 15,
 ('21.4743695', '-157.9777679'): 7,
 ('47.43333', '-70.01667'): 29,
 ('22.1250826', '-159.6025085'): 25,
 ('39.4727008', '-123.8042021'): 11,
 ('43.1191394', '-70.8767004'): 46,
 ('40.5108301', '-74.2097712'): 32,
 ('48.1373395', '-122.565794'): 19,
 ('21.9486586', '-159.371

# Global big day

In [128]:
engine_path = "/Volumes/seagate-storage/db/ebird_small.db"
table_name = 'ebird_small_20190328'
lifelist_filename = 'blank-lifelist.csv'
month = 5
day = 4
padding = 10
num_spots = 5
radius_miles = 5

big_day = chaser(
    engine_path,
    table_name,
    lifelist_filename,
    month,
    day,
    padding,
    num_spots,
    radius_miles
)

big_day

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.


Top spot 1: ('41.7525432', '-81.2912294')
Number possible targets: 206
Most common targets: ['Northern Cardinal', 'Common Grackle', 'Song Sparrow', 'Yellow Warbler', 'Gray Catbird', 'House Wren', 'Northern Flicker', 'Red-winged Blackbird', 'American Goldfinch', 'American Robin', 'American Crow', 'Ring-billed Gull', 'Barn Swallow', 'Blue Jay', 'Baltimore Oriole', 'European Starling', 'Brown Thrasher', 'Killdeer', 'Brown-headed Cowbird', 'Common Yellowthroat', 'House Sparrow', 'Eastern Towhee', 'Yellow-rumped Warbler', 'Wood Thrush', 'Downy Woodpecker', 'Mourning Dove', 'Black-capped Chickadee', 'Chimney Swift', 'Warbling Vireo', 'White-crowned Sparrow', 'Tree Swallow', 'Spotted Sandpiper', 'Tufted Titmouse', 'White-throated Sparrow', 'Herring Gull', 'Red-headed Woodpecker', 'Great Blue Heron', 'Palm Warbler', 'Ruby-crowned Kinglet', 'Chipping Sparrow', 'Northern Rough-winged Swallow', 'Mallard', 'American Redstart', 'Purple Martin', 'Canada Goose', 'Black-and-white Warbler', 'Eastern Ki

{('44.5933032', '-124.39888'): 11,
 ('46.8341927', '-71.0290924'): 5,
 ('47.6634667', '-122.4218833'): 38,
 ('41.9351042', '-82.5104141'): 145,
 ('40.6233341', '-73.5014534'): 12,
 ('36.9182984', '-76.0542297'): 37,
 ('46.2566183', '-72.6331902'): 40,
 ('65.2365', '-126.6939'): 21,
 ('43.8993331', '-77.2260182'): 22,
 ('21.9549272', '-159.335736'): 2,
 ('21.8805358', '-159.4744492'): 1,
 ('43.3130395', '-77.7153471'): 115,
 ('42.0839551', '-70.0490427'): 1,
 ('44.8342', '-87.377'): 21,
 ('37.3578012', '-122.4006665'): 1,
 ('47.9404107', '-69.5062923'): 70,
 ('41.0352234', '-124.1231918'): 3,
 ('41.9435608', '-87.636194'): 74,
 ('46.8', '-71.18333'): 29,
 ('46.2722222', '-72.6111111'): 40,
 ('43.219329', '-79.2162108'): 58,
 ('44.6156447', '-124.0665436'): 6,
 ('29.4833333', '-94.5166667'): 17,
 ('38.005', '-74.474444'): 4,
 ('39.4727008', '-123.8042021'): 36,
 ('41.7525432', '-81.2912294'): 206,
 ('49.3071194', '-123.0042861'): 14,
 ('37.7869181', '-122.499596'): 3,
 ('49.2957439', '-1

# All days, T

In [112]:
engine_path = "/Volumes/seagate-storage/db/ebird_small.db"
table_name = 'ebird_small_20190328'
lifelist_filename = 'tessa-lifelist.csv'
month = 7
day = 2
padding = 184
num_spots = 5
radius_miles = 100

tessa_scores_all = chaser(
    engine_path,
    table_name,
    lifelist_filename,
    month,
    day,
    padding,
    num_spots,
    radius_miles
)

tessa_scores_all

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.


Top spot 1: ('32.2212741', '-110.9281611')
Number possible targets: 21
Most common targets: ['Rufous-winged Sparrow', 'Mexican Whip-poor-will', "Botteri's Sparrow", 'Common Poorwill', 'Ladder-backed Woodpecker', 'Bridled Titmouse', "Cassin's Sparrow", 'Whiskered Screech-Owl', 'Greater Roadrunner', "Lucy's Warbler", "Scott's Oriole", "Abert's Towhee", 'Canyon Towhee', 'Black-throated Sparrow', 'Elf Owl', 'Phainopepla', 'Black-chinned Hummingbird', 'Cactus Wren', 'Arizona Woodpecker', 'Varied Bunting', 'Broad-billed Hummingbird']

Top spot 2: ('31.7310142', '-110.881319')
Number possible targets: 21
Most common targets: ['Rufous-winged Sparrow', 'Mexican Whip-poor-will', "Botteri's Sparrow", 'Common Poorwill', 'Ladder-backed Woodpecker', 'Bridled Titmouse', "Cassin's Sparrow", 'Whiskered Screech-Owl', 'Greater Roadrunner', "Lucy's Warbler", "Scott's Oriole", "Abert's Towhee", 'Canyon Towhee', 'Black-throated Sparrow', 'Elf Owl', 'Phainopepla', 'Black-chinned Hummingbird', 'Cactus Wren', 

{('44.5660126', '-67.056427'): 10,
 ('42.0839551', '-70.0490427'): 8,
 ('42.687714', '-70.6204365'): 7,
 ('60.3413913', '-147.1028137'): 6,
 ('42.6251904', '-70.3616729'): 8,
 ('35.1678', '-120.69428'): 1,
 ('52.0503333', '178.3339833'): 2,
 ('43.8982326', '-69.4348598'): 2,
 ('36.314222', '-121.8902827'): 1,
 ('39.4727008', '-123.8042021'): 3,
 ('58.3287533', '-134.6187401'): 4,
 ('44.6156447', '-124.0665436'): 7,
 ('57.4212944', '-150.1391602'): 1,
 ('70.1355399', '-143.54774'): 7,
 ('48.5922568', '-123.0304022'): 6,
 ('70.3123202', '-148.3125114'): 11,
 ('59.6359965', '-151.5139532'): 4,
 ('42.992209', '-70.7073972'): 7,
 ('32.429324', '-80.1469607'): 1,
 ('38.3044865', '-75.1076889'): 4,
 ('56.6521699', '-132.9288646'): 3,
 ('30.2091599', '-88.9791107'): 4,
 ('57.4991259', '-135.2492523'): 2,
 ('59.9210867', '-148.0890083'): 6,
 ('24.545713', '-81.8121761'): 4,
 ('56.0135866', '-132.6815102'): 3,
 ('59.5253333', '-178.4236667'): 6,
 ('45.103577', '-67.1306705'): 10,
 ('57.2020384',

# All days, B

In [93]:
engine_path = "/Volumes/seagate-storage/db/ebird_small.db"
table_name = 'ebird_small_20190328'
lifelist_filename = 'bryan-lifelist.csv'
month = 7
day = 2
padding = 184
num_spots = 5
radius_miles = 100

bryan_scores_all = chaser(
    engine_path,
    table_name,
    lifelist_filename,
    month,
    day,
    padding,
    num_spots,
    radius_miles
)

bryan_scores_all

Top spot 1: ('44.5660126', '-67.056427')
Number possible targets: 6
Most common targets: ['Atlantic Puffin', 'Manx Shearwater', "Wilson's Storm-Petrel", 'Razorbill', 'Black Guillemot', 'Great Shearwater']

Top spot 2: ('42.992209', '-70.7073972')
Number possible targets: 4
Most common targets: ["Wilson's Storm-Petrel", 'Great Shearwater', 'Roseate Tern', 'Manx Shearwater']

Top spot 3: ('42.9736526', '-70.7647414')
Number possible targets: 4
Most common targets: ["Wilson's Storm-Petrel", 'Great Shearwater', 'Roseate Tern', 'Manx Shearwater']

Top spot 4: ('42.687714', '-70.6204365')
Number possible targets: 4
Most common targets: ["Wilson's Storm-Petrel", 'Great Shearwater', 'Manx Shearwater', 'Roseate Tern']

Top spot 5: ('42.6251904', '-70.3616729')
Number possible targets: 4
Most common targets: ["Wilson's Storm-Petrel", 'Great Shearwater', 'Manx Shearwater', 'Roseate Tern']



/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.


{('44.5660126', '-67.056427'): 6,
 ('42.0839551', '-70.0490427'): 4,
 ('42.687714', '-70.6204365'): 4,
 ('42.6251904', '-70.3616729'): 4,
 ('43.8982326', '-69.4348598'): 1,
 ('57.4212944', '-150.1391602'): 1,
 ('42.992209', '-70.7073972'): 4,
 ('32.429324', '-80.1469607'): 1,
 ('38.3044865', '-75.1076889'): 2,
 ('70.3123202', '-148.3125114'): 2,
 ('59.5253333', '-178.4236667'): 1,
 ('41.4759174', '-82.1756744'): 1,
 ('40.8429942', '-72.4762005'): 2,
 ('37.5534237', '-121.5735054'): 1,
 ('40.466667', '-67.7'): 1,
 ('44.0893', '-123.2945'): 1,
 ('25.77428', '-80.19367'): 1,
 ('38.8904984', '-122.3169923'): 1,
 ('24.6804014', '-81.4071465'): 1,
 ('44.00178', '-72.9296'): 1,
 ('71.38083', '-156.501'): 1,
 ('43.8729007', '-74.4128036'): 1,
 ('43.9852', '-72.9372'): 1,
 ('28.5753263', '-80.9966826'): 1,
 ('42.9736526', '-70.7647414'): 4,
 ('37.2521234', '-119.1801352'): 1,
 ('40.8538774', '-124.1329336'): 1}

In [60]:
my_scores

{('42.6251904', '-70.3616729'): ['Manx Shearwater',
  "Wilson's Storm-Petrel",
  'Pomarine Jaeger',
  'Great Shearwater',
  'Northern Gannet',
  'Common Eider',
  "Cory's Shearwater",
  "Leach's Storm-Petrel",
  'Parasitic Jaeger',
  'Black-legged Kittiwake',
  'Roseate Tern',
  'Little Gull'],
 ('42.687714', '-70.6204365'): ["Cory's Shearwater",
  'Great Shearwater',
  'Parasitic Jaeger',
  "Wilson's Storm-Petrel",
  'Roseate Tern',
  'Northern Gannet',
  'Manx Shearwater',
  'Black-legged Kittiwake',
  'Atlantic Puffin',
  'Black Guillemot',
  "Leach's Storm-Petrel",
  'Little Gull',
  'Common Eider',
  'Pomarine Jaeger'],
 ('40.00081', '-69.61919'): ['Great Shearwater', "Wilson's Storm-Petrel"],
 ('60.3062623', '-172.1920681'): ['Short-tailed Shearwater',
  'Northern Fulmar',
  'Black-legged Kittiwake',
  'Glaucous Gull',
  'Crested Auklet'],
 ('40.70982', '-69.4658'): ['Manx Shearwater',
  "Cory's Shearwater",
  "Wilson's Storm-Petrel",
  'Great Shearwater',
  'Long-tailed Jaeger']

# Entire DB

In [ ]:
assert(False) # Stop kernel from getting to here if "run all cells" is accidentally selected :P

In [ ]:
# File to read from & number of lines to read
#data_file = '/Volumes/seagate-storage/code/ebird_data/medium_dataset.txt']
large_data_file = '/Volumes/seagate-storage/code/ebird_data/ebd_relJan-2019.txt'

# Which columns to use
desired_cols = desired_cols_chaser
                          
# Of columns, which are dates?
date_cols = ['OBSERVATION DATE']

# Chunk size to read through data_file
chunk_size = 100000

# Desired path for SQLITE db; if does not exist, will be created
engine_path = "/Volumes/seagate-storage/db/ebird_chaser.db"

# Name of SQL table
table_name = 'ebird_small_20190328'

# How many chunks to print after
# Should print ~11 times, once every 50*100000 = 50000000 lines
X = 50

engine = process_data(
    engine_path,
    large_data_file,
    desired_cols,
    date_cols,
    chunk_size,
    table_name,
    X
)

# Next steps
* Test amount of time for each type of access of dataset/pandas usage/etc
* Store entire DB (or just USA, Canada?)
* Do chaser 

In [ ]:
def process_data(
    engine_path,
    data_path,
    desired_cols,
    date_cols,
    chunk_size,
    table_name,
    X,
    delete_old_table = True
):
    t0 = time()
    
    engine = create_engine('sqlite:///' + engine_path)
    # If database doesn't already exist, make a new engine
    if not database_exists(engine.url):
        create_database(engine.url)
    # Otherwise, make sure the engine is clear
    # Note that it's MUCH faster to delete this from filesystem directly
    elif delete_old_table:
        meta = MetaData(engine)
        meta.reflect() # Find all tables
        # Only drop the table we're trying to refill
        for tbl in reversed(meta.sorted_tables):
            if tbl.name == table_name:
                engine.execute(tbl.delete())
                print(f'Dropped table {table_name}')
        
    t1 = time()
    
    lines_saved_to_db = 0
    counter = 0

    for chunk in pd.read_csv(
        filepath_or_buffer = data_path, 
        sep = '\t',
        usecols = desired_cols.keys(),
        parse_dates = date_cols, 
        chunksize = chunk_size,
        dtype = desired_cols,
        na_values = ['X']
    ):

        chunk.to_sql(
            name = table_name,
            con = engine,
            if_exists = 'append'
        )

        # Print info every X iterations
        # i.e., every chunk_size * X lines
        lines_saved_to_db += chunk.shape[0]
        if (not counter % X):
            print(f"Iteration {counter}:")
            print(f"  Lines saved to DB: {lines_saved_to_db}") #Helpful for only saving year data
            print(f"  Minutes elapsed: {(time()-t1)/60}\n")
        counter += 1

    db_time = (time()-t1)/60
    total_time = (time()-t0)/60
    estimated_time = ((total_time-db_time) +  (577335302/lines_saved_to_db)*db_time)/60
    print(f"Total iterations: {counter}")
    print(f"Total lines saved to DB: {lines_saved_to_db}") #Helpful for only saving year data
    print(f"Minutes elapsed during DB functions: {db_time}")
    print(f"Total minutes elapsed: {total_time}")
    print(f"Estimated time for all lines: {estimated_time} hours")
    return engine
            
engine = process_data(
    engine_path,
    data_file,
    desired_cols,
    date_cols,
    chunk_size,
    table_name,
    X
)


### Old code that might help with next steps

In [ ]:
num_lines = 577335302 #from previous analysis
chunk_size = 1000000 # number of lines to read at once
num_chunks = 3 # number of chunks to test on

In [ ]:
the_cols = [
        'Last edited date', 
        'category', 
        'common name', 
        'scientific name', 
        'observation count', 
        'country', 
        'state', 
        'latitude', 
        'longitude', 
        'observation date', 
        'observer id', 
        'protocol type',
        'ALL SPECIES REPORTED',
]

desired_cols = [item.upper() for item in the_cols]
desired_cols

In [ ]:
just_byemeh = pd.DataFrame({'hi':[0, 1, 2], 'bye':[8, 9, 9], 'meh':[42, 42, 20]}, index=[0, 1, 2])[['bye', 'meh']]
just_byemeh.loc[just_byemeh['meh'] == 42]

just_byemeh.loc[
    (just_byemeh['meh'] == 42) & 
    (just_byemeh['bye'] == 9)]

In [ ]:

def save_to_db(chunk, table_name, engine):
    '''
    Process & save data from chunk
    
    Processes data to identify
    
    Inputs:
        chunk: a dataframe to save
        table_name:
        engine: SQLAlchemy connection to use
    '''

    
    # For using only 2018 data:
    #only_2018 = chunk.loc[
    #    (chunk['LAST EDITED DATE'].dt.year == 2018) &
    #    (chunk['OBSERVATION DATE'].dt.year == 2018)]
    #if only_2018.shape[0]: # if this DF has anything in it (i.e. any 2018 records)
    #    only_2018.to_sql('ebird_data_2018', engine, if_exists='append')
    #return only_2018

engine = create_engine("sqlite:////Volumes/seagate-storage/db/ebird_2018.db")
if not database_exists(engine.url):
    create_database(engine.url)
    
    
    
the_cols = [
        'Last edited date', 
        'category', 
        'common name', 
        'scientific name', 
        'observation count', 
        'country', 
        'state', 
        'latitude', 
        'longitude', 
        'observation date', 
        'observer id', 
        'protocol type',
        'ALL SPECIES REPORTED',
]

desired_cols = [item.upper() for item in the_cols]
desired_cols


data_file = '/Volumes/seagate-storage/code/evil_project/example_dataset.txt' #'/Volumes/seagate-storage/code/ebird_data/ebd_relJan-2019.txt'


num_lines = 577335302 #from previous analysis
chunk_size = 1000000 # number of lines to read at once


def process(chunk):
    
    only_2018 = chunk.loc[
        (chunk['LAST EDITED DATE'].dt.year == 2018) &
        (chunk['OBSERVATION DATE'].dt.year == 2018)]
    
    if only_2018.shape[0]: # if this DF has anything in it
        only_2018.to_sql('ebird_data_2018', engine, if_exists='append')

    return only_2018

t1 = time.time()

lines_saved_to_db = 0
counter = 0

for chunk in pd.read_csv(
    data_file, 
    sep='\t', 
    #index_col = ['SAMPLING EVENT IDENTIFIER'],
    usecols = desired_cols,
    parse_dates = ['LAST EDITED DATE', 'OBSERVATION DATE'], 
    chunksize=chunk_size):
    
    '''
    '''
    
    finished_chunk = process(chunk)
    lines_saved_to_db += finished_chunk.shape[0]

    if not counter % 100:
        print(f"Iteration {counter}:")
        print(f"  Lines saved to DB: {lines_saved_to_db}")
        print(f"  Minutes elapsed: {(time.time()-t1)/60}")
    counter += 1
    
    
t2 = time.time()

process_time = t2-t1
print('Time to process rows:', process_time)

#est_time = (num_lines/(chunk_size*num_chunks))*process_time
#print('Estimated time (seconds) for all rows:', est_time)

Reading the whole DF:

```
chunk_size = 1000000 # number of lines to read at once
num_chunks = 3 # number of chunks to test on

(1000000, 46)
(1000000, 46)
(1000000, 46)
Time to process 3 rows: 23.173866748809814
Estimated time for all rows: 4459.697119310624


---

chunk_size = 3000000 # number of lines to read at once
num_chunks = 1 # number of chunks to test on

(3000000, 46)
Time to process rows: 23.521601915359497
Estimated time (seconds) for all rows: 4526.6170484426175

---

chunk_size = 500000 # number of lines to read at once
num_chunks = 6 # number of chunks to test on

(500000, 46)
(500000, 46)
(500000, 46)
(500000, 46)
(500000, 46)
(500000, 46)
Time to process rows: 24.359390020370483
Estimated time (seconds) for all rows: 4687.845264648793
```

Reading only desired columns, with `parse_dates`
```
(500000, 12)
(500000, 12)
(500000, 12)
(500000, 12)
(500000, 12)
(500000, 12)
Time to process rows: 13.444914102554321
Estimated time (seconds) for all rows: 2587.407847920753
```

Reading only desired columns, without `parse_dates`:

```
(500000, 12)
(500000, 12)
(500000, 12)
(500000, 12)
(500000, 12)
(500000, 12)
Time to process rows: 11.651726961135864
Estimated time (seconds) for all rows: 2242.3177679763053
```


In [ ]:
pd.read_csv(small_data_file, sep='\t').to_sql('test_ebird_data', engine)

In [ ]:
import pandas as pd
small_data_file = '/Volumes/seagate-storage/code/ebird_data/test_dataset.txt'
pd.read_csv(small_data_file, sep='\t').to_sql('test_ebird_data', engine)

# Clean up data

In [ ]:
import time
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import pandas as pd
from functools import wraps
from time import time

In [ ]:
small_db = pd.read_sql('test_ebird_data',con=engine)

In [ ]:
small_db.columns

### Keep only the needed columns

In [ ]:
def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print('func took: {} sec'.format(te-ts))
        return result
    return wrap


def process_chunk(chunk_df):
    new_df = chunk_df[['COMMON NAME','LATITUDE', 'LONGITUDE', 'OBSERVATION DATE']]
    new_df.to_sql('med_occ', medium_engine, if_exists='append')

@timing
def process_all_data(engine_path, data_path):
    engine = create_engine('sqlite:///'+engine_path)
    for chunk in pd.read_csv(data_path, sep='\t', chunksize=100):
        process_chunk(chunk)


process_all_data(
    engine_path='/Volumes/seagate-storage/db/large_ebird.db', 
    data_path='/Volumes/seagate-storage/code/ebird_data/large_dataset.txt')

In [ ]:
small_database = database[['COMMON NAME','LATITUDE', 'LONGITUDE', 'OBSERVATION DATE']]
small_database